Use a CNN to classify trimmed spectra


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [46]:
# Read in individual datacubes and combine into a single dataframe

from astropy.table import Table
import pandas as pd

df = pd.DataFrame()

sightline_list = ['J010619+004823','J012403+004432','J013340+040059','J013724-422417','J015741-010629',
                  'J020944+051713','J024401-013403','J033413-161205','J033900-013318','J094932+033531',
                  'J095852+120245','J102009+104002','J111008+024458','J111113-080401','J120917+113830',
                  'J123055-113909','J124957-015928','J133254+005250','J142438+225600','J162116-004251',
                  'J193957-100241','J200324-325145','J205344-354652','J221527-161133','J230301-093930',
                  'J231543+145606','J233446-090812','J234913-371259']

for sightline in sightline_list:
    table = Table.read('/home/emma/Documents/laeML/data/catalogues/individual_cubes/{}_cubex_data.fits'.format(sightline))
    table['sightline'] = sightline
    pandas_df = table.to_pandas()
    df = pd.concat([df,pandas_df])

In [47]:
# Cut to SNR > 7 as we haven't looked at all the sources below this
df = df[df.SNR >= 7]

In [48]:
# Check number of LAEs
type_str = df['type'].str.decode("utf-8")
nLAE = type_str.str.contains('LAE').sum()
print(nLAE)

1013


In [49]:
# Create new column to show if entry is an LAE
df['isLAE'] = np.where(type_str.str.contains('LAE'), 1, 0)

We want to use 1D flux arrays from the trimmed spectra as input to our CNN (i.e. X) while is_LAE is the target (i.e. Y)

In [50]:
y=df.isLAE

Need to read in the flux columns from the trimmed arrays and combine into a dataframe to use as X

In [7]:
#instead of creating x from each file individually, read it in from a prepared file
xAll=pd.read_csv("/home/emma/Documents/laeML/data/trimmed_spectra_dataframe.csv",index_col=0)

In [8]:
xAll.head()

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,J010619+004823,5,590.384766,454.941467,354.345154,328.243042,5.348419,70.211662,235.562622,207.469299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J010619+004823,9,452.941223,427.720490,435.023926,538.362366,448.548431,518.894287,402.824951,319.419006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J010619+004823,18,237.862305,131.504059,196.990021,375.919373,264.392822,225.224884,189.368408,158.257416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,J010619+004823,703,371.443054,337.741241,242.179245,256.283417,389.117188,477.863403,372.696228,448.540527,...,290.749451,296.189941,207.545380,299.615936,240.926071,307.433838,527.028931,481.803894,500.537750,474.636536
4,J010619+004823,784,47.715321,-27.427366,42.573402,34.838043,89.349220,-45.284454,-17.226364,-45.073406,...,114.334274,35.456001,-106.346481,25.491888,45.191193,45.270767,-9.925606,-58.586670,39.541794,29.653870


In [9]:
x=xAll.drop(['0','1'], axis=1)

In [10]:
x.head()

,2,3,4,5,6,7,8,9,10,11,...,152,153,154,155,156,157,158,159,160,161
0,590.384766,454.941467,354.345154,328.243042,5.348419,70.211662,235.562622,207.469299,137.152069,109.381187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,452.941223,427.720490,435.023926,538.362366,448.548431,518.894287,402.824951,319.419006,319.533020,267.410187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,237.862305,131.504059,196.990021,375.919373,264.392822,225.224884,189.368408,158.257416,97.674271,99.918762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,371.443054,337.741241,242.179245,256.283417,389.117188,477.863403,372.696228,448.540527,375.683411,559.601074,...,290.749451,296.189941,207.545380,299.615936,240.926071,307.433838,527.028931,481.803894,500.537750,474.636536
4,47.715321,-27.427366,42.573402,34.838043,89.349220,-45.284454,-17.226364,-45.073406,23.561562,-54.283142,...,114.334274,35.456001,-106.346481,25.491888,45.191193,45.270767,-9.925606,-58.586670,39.541794,29.653870


In [11]:
x=x.fillna(0)

In [12]:
x = np.array(x).reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(15356, 160, 1)


In [35]:
#create the final test/validation sample that the model never sees
from sklearn.model_selection import train_test_split
x, x_final, y, y_final=train_test_split(x, y, test_size=0.1)

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.15)

In [37]:
nLAE_trainSample = len(y_train[y_train==1])
print(nLAE_trainSample)

285


In [38]:
#Create simple CNN

#import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(160,1)))
model.add(MaxPooling1D())
model.add(Conv1D(64, 2, activation="relu"))
model.add(MaxPooling1D())
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))


In [39]:
model.compile(loss = 'binary_crossentropy', 
     optimizer = "adam",               
    metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 159, 64)           192       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 79, 64)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 78, 64)            8256      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 39, 64)           0         
 1D)                                                             
                                                                 
 dense_2 (Dense)             (None, 39, 16)            1040      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 19, 16)          

In [40]:
model.fit(x_train, y_train,
          batch_size=200,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
53/53 [==============================] - 3s 38ms/step - loss: 0.5751 - accuracy: 0.9591 - val_loss: 0.1950 - val_accuracy: 0.9695
Epoch 2/20
53/53 [==============================] - 2s 37ms/step - loss: 0.1562 - accuracy: 0.9657 - val_loss: 0.1167 - val_accuracy: 0.9625
Epoch 3/20
53/53 [==============================] - 2s 36ms/step - loss: 0.1093 - accuracy: 0.9700 - val_loss: 0.1072 - val_accuracy: 0.9657
Epoch 4/20
53/53 [==============================] - 2s 37ms/step - loss: 0.1029 - accuracy: 0.9707 - val_loss: 0.0995 - val_accuracy: 0.9700
Epoch 5/20
53/53 [==============================] - 2s 36ms/step - loss: 0.0963 - accuracy: 0.9713 - val_loss: 0.1392 - val_accuracy: 0.9705
Epoch 6/20
53/53 [==============================] - 2s 35ms/step - loss: 0.1053 - accuracy: 0.9710 - val_loss: 0.1015 - val_accuracy: 0.9700
Epoch 7/20
53/53 [==============================] - 2s 36ms/step - loss: 0.0911 - accuracy: 0.9711 - val_loss: 0.0952 - val_accuracy: 0.9705
Epoch 8/20
53

In [41]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.09076045453548431
Test accuracy: 0.9694533944129944


In [42]:
ypred = model.predict(x_final)
ypred = np.array(ypred)

for yi in range(len(ypred)):
    if ypred[yi]<0.5:
        ypred[yi] = 0
    else:
        ypred[yi] = 1

44/44 [==============================] - 0s 2ms/step


In [43]:
y_final=np.array(y_final)
nLAE_finalsample = len(y_final[y_final==1])
print(nLAE_finalsample)

29


In [51]:
isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE, NotLAE_and_NotpredLAE = 0,0,0,0

for i in range(len(y_final)):
    if (y_final[i] == 1) & (ypred[i][0]==1):
        isLAE_and_predLAE += 1 # Number of True LAEs predicted to be LAEs
    elif (y_final[i] == 1) & (ypred[i][0]!=1):
        isLAE_and_NotpredLAE += 1 # Number of True LAEs not predicted to be LAEs
    if (y_final[i] != 1) & (ypred[i][0]==1):
        NotLAE_and_predLAE += 1 # Number of non-LAEs predicted to be LAEs
    if (y_final[i] != 1) & (ypred[i][0] != 1):
        NotLAE_and_NotpredLAE += 1 # Number of non-LAEs predicted to not be LAEs

In [52]:
print(isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE,NotLAE_and_NotpredLAE)

4 25 15 1338
